In [2]:
from tqdm import tqdm
import numpy as np

import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. CartPole environment

In [3]:
""" Environment Information
ref: https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py#L17

Action Space
    The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
     of the fixed force the cart is pushed with.
    | Num | Action                 |
    |-----|------------------------|
    | 0   | Push cart to the left  |
    | 1   | Push cart to the right |
    **Note**: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle
     the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it
Observation Space
    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:
    | Num | Observation           | Min                 | Max               |
    |-----|-----------------------|---------------------|-------------------|
    | 0   | Cart Position         | -4.8                | 4.8               |
    | 1   | Cart Velocity         | -Inf                | Inf               |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
    | 3   | Pole Angular Velocity | -Inf                | Inf               |
    **Note:** While the ranges above denote the possible values for observation space of each element,
        it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:
    -  The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates
       if the cart leaves the `(-2.4, 2.4)` range.
    -  The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates
       if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)
Rewards
    Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
    including the termination step, is allotted. The threshold for rewards is 475 for v1.
Episode End
    The episode ends if any one of the following occurs:
    1. Termination: Pole Angle is greater than ±12°
    2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
    3. Truncation: Episode length is greater than 500 (200 for v0)
"""
env = gym.make("CartPole-v1")
# env.reset()
# env.render()

Sample environment image

<img width=300 src="cartpole.png" />

In [4]:
print("observation_space: ", env.observation_space)
print("action_space: ", env.action_space)
state = env.reset()
print("sample obs: ", state)

observation_space:  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action_space:  Discrete(2)
sample obs:  [-0.04331493  0.01159795 -0.01566737  0.01882051]


In [5]:
# sample step
new_state, reward, done, info = env.step(0)
print("sample step: ", (new_state, reward, done, info))

sample step:  (array([-0.04308297, -0.18329585, -0.01529096,  0.3065193 ], dtype=float32), 1.0, False, {})


# 2. REINFORCE: Vanilla Policy Gradient

In [ ]:
class VanillaNN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(VanillaNN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=-1)  # dim need to be -1 to prevent NaN results
        return x

In [ ]:
class Reinforce:
    def __init__(self, env, gamma=0.99, learning_rate=3e-3, device="cpu"):
        self.env = env
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.device = device

        # get number of actions and observations
        self._n_observations = self.env.observation_space.shape[0]
        self._n_actions = self.env.action_space.n

        # setup NN model
        self.policy_net = VanillaNN(self._n_observations, self._n_actions).to(self.device)

        # setup optimizer and loss function
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.criterion = self.loss_function
        
    def loss_function(self, prob_batch, expected_returns_batch):
        return - torch.sum(torch.log(prob_batch) * expected_returns_batch)

    def get_trajectory(self, max_steps=500, render=False):
        trajectory = list()
        done = False  # incase of early loop-termination (max_steps) before the environment terminated
        state = self.env.reset()

        for _ in range(max_steps):
            action_probs = self.policy_net(torch.from_numpy(state).float())
            # TODO: support continuous action space
            selected_action = np.random.choice(np.array(range(self._n_actions)), p=action_probs.data.numpy())
            next_state, reward, done, _ = self.env.step(selected_action)
            trajectory.append((state, selected_action, reward))
            state = next_state
            
            if render:
                self.env.render()

            if done:
                break

        return trajectory, done

    def train(self, num_episodes=1000, max_steps=500):
        for episode in tqdm(range(num_episodes)):
            # get sample trajectory from the policy network
            trajectory, _ = self.get_trajectory(max_steps=max_steps)
            
            # prepare data for training
            states, actions, rewards = zip(*trajectory)
            states = torch.Tensor(states)
            actions = torch.Tensor(actions)
            
            # calculate expected discounted returns
            expected_returns_batch = list()
            for idx in range(len(rewards)):
                discounted_rewards = [self.gamma**i * reward for i, reward in enumerate(rewards[idx:])]
                expected_returns_batch.append(sum(discounted_rewards))
            
            # normalize and reformat expected returns
            expected_returns_batch = torch.FloatTensor(expected_returns_batch)
            expected_returns_batch /= expected_returns_batch.max()
            
            # calculate loss
            action_probs = self.policy_net(states)
            prob_batch = action_probs.gather(dim=1,index=actions.long().view(-1,1)).squeeze() 
            loss = self.criterion(prob_batch, expected_returns_batch)
            
            # optimize the model with backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
    def visualize_policy(self, num_episodes=1, max_steps=500):
        for _ in range(num_episodes):
            trajectory, done = self.get_trajectory(max_steps=max_steps, render=True)
            
    def evaluate_policy(self, num_episodes=100, max_steps=500):
        win = 0
        for _ in range(num_episodes):
            trajectory, done = self.get_trajectory(max_steps=max_steps)
            # CUSTOMIZE: for lunarlander environment
            if done and trajectory[-1][2] == 100:
                win += 1
        return win / num_episodes

In [70]:
env = gym.make("CartPole-v1")
reinforce_agent = Reinforce(env)
reinforce_agent.train(num_episodes=int(500), max_steps=200)

100%|██████████| 500/500 [00:03<00:00, 134.83it/s]


In [76]:
# test the trained agent in 1 episode
trajectory, done = reinforce_agent.get_trajectory(max_steps=501)
print("done status: ", done)
print("step taken (maximum 500): ", len(trajectory))

done status:  True
step taken (maximum 500):  500


In [77]:
# test the trained agent
def evaluate_policy(agent, num_episodes=100, max_steps=500):
    win = 0
    for _ in range(num_episodes):
        trajectory, _ = agent.get_trajectory(max_steps=max_steps)
        # CUSTOMIZE: for cartpole environment
        if len(trajectory) >= 490:
            win += 1
    return win / num_episodes

test_episodes = 100
win_rate = evaluate_policy(reinforce_agent, num_episodes=test_episodes)
print(f'agent win rate: {win_rate*100 :.2f}% from {test_episodes} test episodes')

agent win rate: 98.00% from 100 test episodes


In [80]:
# visualize the trained agent on a separate window
reinforce_agent.visualize_policy(num_episodes=10)